In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from altair import datum
import ast
from datetime import datetime


# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable('data_server')
# # Include an image for each plot since Gradescope only supports displaying plots as images
# alt.renderers.enable('mimetype')

In [ ]:
url = "https://github.com/kemiolamudzengi/dsci-320-datasets/blob/main/amazon_conlit_goodreads_nyt.csv?raw=true"
books = pd.read_csv(url, parse_dates=['amazon_year', 'conlit_pubdate', 'nyt_published_date',
                                      'goodreads_publish_date', 'goodreads_first_publish_date'
                                     ] )

In [ ]:
books['conlit_pubyear'] = books['conlit_pubdate'].dt.year

dropdown_options = sorted(
    books['conlit_pubyear'].dropna().unique(), 
    reverse=True
)

year_selector = alt.binding_select(
    options=dropdown_options,
    name = 'Year: '
)
year_dropdown = alt.selection_single(
    fields=['conlit_pubyear'], 
    bind=year_selector
)
## line chart over time

genre_line = alt.Chart(books.dropna(subset=['conlit_genre'])).mark_line().encode(
    alt.X('year(conlit_pubdate):O'),
    alt.Y('count()'),
    alt.Color('conlit_genre')
).properties(
    width = 600,
    height = 250
)
# bar chart
genre_bar = alt.Chart(books.dropna(subset=['conlit_genre'])).transform_filter(
    year_dropdown
).mark_bar().encode(
    alt.X('count()', axis=alt.Axis(tickMinStep=1)),
    alt.Y('conlit_genre', sort='-x')
).add_selection(
    year_dropdown
).properties(
    width=600,
    height=250
)

In [ ]:
genre_line & genre_bar